IMPORTING THE REQUIRED PACKAGES

In [ ]:
#pip install mediapipe

In [ ]:
import cv2
import mediapipe
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#address of the folder where train dataset of images is saved (done on kaggle notebook)
main_dir = "../input/yogapose/Train/"

In [ ]:
#creating a dictionary of all the 5 yoga poses
DICT = {
    "downdog": 0,
    "goddess":1,       
    "plank":2,
    "tree":3,
    "warrior2":4
}
DICT

{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}

In [ ]:
#defining a method to transfer images into a dataframe 

def getdata(folder_path):

    #making an empty dataframe which will contain image address and to which yoga pose it belongs
    sig = pd.DataFrame(columns=['image_abs_path','image_labels'])
    
    for key,value in DICT.items():
        #print("processing for label: {}".format(label))
        label_i = folder_path+"/"+str(key)
        #read directory
        dirs_label_i =  os.listdir(label_i)
        idx = 0
        for image in dirs_label_i:
            #create an absolute image path
            sig_i = os.path.join(label_i,image)
            #print('Absolute path for image no. {} and label {}: {}'\
                  #.format(idx,label,flower_i))

            #filling the dataframe with path and label
            sig = sig.append({'image_abs_path':sig_i,
                            'image_labels':key},
                           ignore_index=True)
            idx += 1
            
    return sig

In [ ]:
#calling the above function 
train = getdata(main_dir)

train[train["image_labels"]=="downdog"]

,image_abs_path,image_labels
0,../input/yogapose/Train//downdog/00000335.jpg,downdog
1,../input/yogapose/Train//downdog/00000287.png,downdog
2,../input/yogapose/Train//downdog/00000382.jpg,downdog
3,../input/yogapose/Train//downdog/00000233.jpg,downdog
4,../input/yogapose/Train//downdog/00000310.png,downdog
...,...,...
224,../input/yogapose/Train//downdog/00000238.jpg,downdog
225,../input/yogapose/Train//downdog/00000151.jpg,downdog
226,../input/yogapose/Train//downdog/00000142.jpg,downdog
227,../input/yogapose/Train//downdog/00000314.jpg,downdog


In [ ]:
#method to retrive the keypoints for each image and storing it to its respective csv file based on the yoga pose it belongs to. 
def getKeyPts(imgpath):
      
       modelPose = mediapipe.solutions.pose
       #mpDraw = mediapipe.solutions.drawing_utils

       pose = modelPose.Pose()
       mp_holistic = mediapipe.solutions.holistic
      
       img = cv2.imread(imgpath)  
       frameRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
       results = pose.process(frameRGB)

       return results.pose_landmarks

In [ ]:
#listing index of all the keypoints that are being extracted from above method
key_points = ['Nose','Left_eye_inner','Left_eye','Left_eye_outer','right_eye_inner',
              'right_eye','right_eye_outer','left_ear','right_ear','mouth_left',
              'mouth_right','lef_shoulder','right_shoulder','lef_elbow','right_elbow',
              'left_wrist','right_wrist','left_pinky','right_pinky','left_index',
              'right_index','left_thumb','right_thumb','left_hip','right_hip',
              'left_knee','right_knee','left_ankle','right_ankle',
              'left_heel','right_heel','left_foot_index','right_foot_index']

In [ ]:
#main method to save all keypoints of each pimage in respective csv files
#since kaggle was reaching its max memory limit while processing the images so had to set the limit

def yoga(pose,limit):

    global key_points,DICT
    counter = 0
    header = {"pose":[]}
    for key_point in key_points:
        header['{} x'.format(key_point)] = []
        header['{} y'.format(key_point)] = []
        
    ################################################
    #looping for every pose and filering the images for that pose
    for pose in [pose]:
        df = pd.DataFrame(header)
        print('Generating results for {}'.format(pose))
        filtrdata = train[train["image_labels"]==pose]

    ##################################################
        #after getting all the images for 1 pose calling the getkeyPts on each image 
        for row in filtrdata['image_abs_path']:
            if counter <= limit:
                print(f"Count {counter} : {row}")

                keypoints = getKeyPts(row)
                new_row = pd.DataFrame(header)
                #using try & exeption block so if the keypoint cannot be detected it doesnt throw an error instead print the error and continue
                try:
                    for id,keyPts in enumerate(keypoints.landmark):
                        #print(id,keyPts)
                        #extracting only the x and y coordinates of each keypoint on the image and
                        cx,cy = (keyPts.x*1),(keyPts.y*1)
                        #print(f"({cx},{cy}) for {id}")
                        #print(key_points[id])
                        #print(f"({cx},{cy})")
                        #saving the x and y coords in the new row datadrame
                        new_row['{} x'.format(key_points[id])] = [cx]
                        new_row['{} y'.format(key_points[id])] = [cy]
                        #print(new_row)
                    #appending the pose the image belongs to at the end of each row
                    new_row['pose'] = pose

                    #appending each new_row df to final dataframe of that yoga pose
                    df = df.append(new_row, ignore_index=True)
                    #print(df)
                    counter += 1

                except Exception as e:
                    print('Exception on {}! {}'.format(row, e))
            else:
                continue
        print("Saving csv file.....")
        
        #saving the data frame to csv for each pose
        df.to_csv('{}.csv'.format(pose))
        counter = 0

In [ ]:
yoga(pose="downdog",limit=50)

Generating results for downdog
Count 0 : ../input/yogapose/Train//downdog/00000335.jpg
Count 1 : ../input/yogapose/Train//downdog/00000287.png
Count 2 : ../input/yogapose/Train//downdog/00000382.jpg
Count 3 : ../input/yogapose/Train//downdog/00000233.jpg
Count 4 : ../input/yogapose/Train//downdog/00000310.png
Count 5 : ../input/yogapose/Train//downdog/00000387.jpg
Exception on ../input/yogapose/Train//downdog/00000387.jpg! 'NoneType' object has no attribute 'landmark'
Count 5 : ../input/yogapose/Train//downdog/00000309.jpg
Count 6 : ../input/yogapose/Train//downdog/00000277.jpg
Count 7 : ../input/yogapose/Train//downdog/00000325.jpg
Count 8 : ../input/yogapose/Train//downdog/00000147.jpg
Exception on ../input/yogapose/Train//downdog/00000147.jpg! 'NoneType' object has no attribute 'landmark'
Count 8 : ../input/yogapose/Train//downdog/00000236.JPG
Count 9 : ../input/yogapose/Train//downdog/00000366.png
Count 10 : ../input/yogapose/Train//downdog/00000199.jpg
Count 11 : ../input/yogapose

In [ ]:
pose = [x for x,y in DICT.items()]
for i in range(1,len(pose)):
    yoga(pose=pose[i],limit=50)

Generating results for goddess
Count 0 : ../input/yogapose/Train//goddess/00000113.jpg
Count 1 : ../input/yogapose/Train//goddess/00000115.jpg
Count 2 : ../input/yogapose/Train//goddess/00000233.jpg
Count 3 : ../input/yogapose/Train//goddess/00000336.jpg
Count 4 : ../input/yogapose/Train//goddess/00000325.jpg
Count 5 : ../input/yogapose/Train//goddess/00000211.jpg
Count 6 : ../input/yogapose/Train//goddess/00000278.png
Count 7 : ../input/yogapose/Train//goddess/00000129.jpg
Count 8 : ../input/yogapose/Train//goddess/00000302.jpg
Count 9 : ../input/yogapose/Train//goddess/00000105.jpg
Count 10 : ../input/yogapose/Train//goddess/00000270.jpg
Count 11 : ../input/yogapose/Train//goddess/00000212.jpg
Count 12 : ../input/yogapose/Train//goddess/00000228.jpg
Exception on ../input/yogapose/Train//goddess/00000228.jpg! 'NoneType' object has no attribute 'landmark'
Count 12 : ../input/yogapose/Train//goddess/00000369.jpg
Count 13 : ../input/yogapose/Train//goddess/00000303.jpg
Count 14 : ../inpu

In [ ]:
yoga(pose="warrior2",limit=50)

Generating results for warrior2
Count 0 : ../input/yogapose/Train//warrior2/00000390.jpg
Count 1 : ../input/yogapose/Train//warrior2/00000127.jpg
Count 2 : ../input/yogapose/Train//warrior2/00000335.jpg
Count 3 : ../input/yogapose/Train//warrior2/00000287.png
Count 4 : ../input/yogapose/Train//warrior2/00000382.jpg
Count 5 : ../input/yogapose/Train//warrior2/00000233.jpg
Count 6 : ../input/yogapose/Train//warrior2/00000429.jpg
Count 7 : ../input/yogapose/Train//warrior2/00000437.jpg
Count 8 : ../input/yogapose/Train//warrior2/00000336.jpg
Count 9 : ../input/yogapose/Train//warrior2/00000387.jpg
Count 10 : ../input/yogapose/Train//warrior2/00000309.jpg
Count 11 : ../input/yogapose/Train//warrior2/00000325.jpg
Count 12 : ../input/yogapose/Train//warrior2/00000253.jpg
Count 13 : ../input/yogapose/Train//warrior2/00000211.jpg
Count 14 : ../input/yogapose/Train//warrior2/00000267.jpg
Count 15 : ../input/yogapose/Train//warrior2/00000147.jpg
Count 16 : ../input/yogapose/Train//warrior2/00000